In [0]:


spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import pandas as pd
pd.set_option('expand_frame_repr', False)
import boto3
from datetime import datetime
import pandas as pd
from pyspark.sql import functions as F
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING
from conf.settings import *
from applications.db_check_v1.common.db_check_utils import query_df
import pandas as pd

daily_est_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
    )
)


def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list


def compare(date):
    sql = """
select distinct(app_id) from plproxy.execute_select_nestloop($proxy$ 
    select distinct(app_id)
    from aa.app_store_daily_estimate_{}
    where 
        date = '{}'
$proxy$) tbl (app_id BIGINT);
"""
    print datetime.today()
    db_df = []
    for device in [0,1,2]:
        db_device_df = query_df(daily_est_dsn, sql.format(device, date))
        db_df = db_device_df if len(db_df)==0 else pd.concat([db_df, db_device_df])
    db_df = db_df.drop_duplicates()
    print datetime.today()
    unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date)).select("app_id").distinct().toPandas()
    print datetime.today()
    
    
    diff_df =db_df.merge(unified_df, indicator=True, how='left')
    diff_df = diff_df.loc[diff_df["_merge"] != "both"]
    
    sql = """
select app_id,feed_id,store_id,estimate from plproxy.execute_select_nestloop($proxy$ 
    select app_id,feed_id,store_id,estimate
    from aa.app_store_daily_estimate_{}
    where 
        date = '{}' AND app_id in ({})
$proxy$) tbl(app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT);
""" 
    print len(diff_df)
    diff_app_id = [str(x) for x in diff_df.app_id.tolist()]
    print sql.format(1, date, ",".join(diff_app_id))
    db_df = query_df(daily_est_dsn, sql.format(1, date, ",".join(diff_app_id)))
    print db_df.store_id.unique()

    
    print datetime.today()
    if len(diff_df) ==  0:
        return "{};PASS".format(date)
    else:
        return "{};FAIL; {} {} {}".format(date, len(unified_df),len(db_df), diff_df.app_id.tolist()[0:5])
        
        
def write_log(strobj, s3obj):
    s3obj.put(Body=str(strobj))


In [0]:

s3 = boto3.resource('s3')
s3object = s3.Object('b2c-prod-data-pipeline-qa', 'tom/all3/regression1.txt')
log = [] 

# 2010-09-02/   2020-02-21
date_list =get_date_list('2019-07-14', '2019-09-10')
for date in date_list:
    temp_log = ""
    # compare(date)
    try:
        temp_log = compare(date)
        print temp_log
    except Exception, e:
        temp_log = "{}; ERROR; {}".format(date, e.message)
        print temp_log
        print 
    log.append(temp_log)
    write_log(log, s3object)

In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 

select app_id,feed_id,store_id,estimate from plproxy.execute_select_nestloop(\$proxy\$ 
    select app_id,feed_id,store_id,estimate
    from aa.app_store_daily_estimate_1
    where 
        date = '2019-09-01' AND app_id in (985896290, 470074597, 497708025, 720245280, 540041963, 391329860, 829641544, 1148468242, 928368430, 859604306, 1046307415 )
\$proxy\$) tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT);

EOF


In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 

select app_id,feed_id,store_id,estimate from plproxy.execute_select_nestloop(\$proxy\$ 
    select app_id,feed_id,store_id,estimate
    from aa.app_store_daily_estimate_1
    where 
        date = '2019-09-01' AND app_id in (985896290, 470074597, 497708025, 720245280, 540041963, 391329860, 829641544, 1148468242, 928368430, 859604306, 1046307415 )
    limit 5
\$proxy\$) tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT);

EOF


In [0]:


def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list

date_list = get_date_list("2019-07-14", "2020-02-24")

for date in date_list:
    df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date))
    unifie_count = df.select("app_id").distinct().count()

    df = spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date={}/".format(date))
    raw_count= df.select("id").distinct().count()
    
    if unifie_count!=raw_count:
        print "{}; Fail; {};{}".format(date, raw_count, unifie_count)
    else:
        print "{}; Pass".format(date)

In [0]:

from pyspark.sql import types as T
import pandas as pd

EST_SCHEMA = T.StructType(
[
    T.StructField("country_code", T.StringType(), True),
    T.StructField("date", T.StringType(), True),
    T.StructField("device_code", T.StringType(), True),
    T.StructField("granularity", T.StringType(), True),
    T.StructField("free_app_download", T.StringType(), True),  
    T.StructField("app_id", T.StringType(), True),
    T.StructField("paid_app_download", T.StringType(), True),
    T.StructField("revenue", T.StringType(), True),
    T.StructField("revenue_iap", T.StringType(), True)    ]
)
    
def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list

date_list = get_date_list("2016-10-24", "2017-12-31")
# date_list = get_date_list("2010-08-02", "2019-07-13")

for date in date_list:
    unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date)).select("app_id").distinct()
    unifie_count = unified_df.count()
    # unified_df.write.parquet("s3://b2c-prod-data-pipeline-qa/tom/store/unified/date={}/".format(date))
    

    raw_df = spark.read.option("delimiter", "\t").csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/*/sbe_est_app/*/".format(date)).select("_c5").distinct()
    raw_count= raw_df.count()
    # raw_df.write.parquet("s3://b2c-prod-data-pipeline-qa/tom/store/raw/date={}/".format(date))

    
    if unifie_count!=raw_count:
        print "{}; Fail; {};{}".format(date, raw_count, unifie_count)
    else:
        print "{}; Pass".format(date)